In [1]:
!pip install findspark
!pip install pyspark
!pip install spark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=1b64e8ac8a84b5863f306f7c4b4e22dac8fcee7f704655fc23a05c884aed5fdc
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for spark: filename=spark-0.2.1-py3-none-any.whl size=58748 sha256=1a71983ac1191cf5e5ae1675f18cbcc4547b0ee178152596515da27190586092
  Stored in directory: /root/.cache/pip/wheels/63/88/77/b4131110ea4094540f7b47c6d62a649807d7e94800da5eab0b
Successfully built spark


In [2]:
import pyspark
import findspark
import spark

findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

In [3]:
departamentos = spark.read.parquet('departamentos.parquet')
empleados = spark.read.parquet('empleados.parquet')

AnalysisException: ignored

# Join

In [83]:
depa = departamentos.withColumn('num_dpto', col('id'))

In [84]:
depa.printSchema()
departamentos.printSchema()

root
 |-- id: long (nullable = true)
 |-- nombre_dpto: string (nullable = true)
 |-- num_dpto: long (nullable = true)

root
 |-- id: long (nullable = true)
 |-- nombre_dpto: string (nullable = true)



In [89]:
empleados.join(depa, empleados.num_dpto == depa.num_dpto).show()

+------+--------+---+-----------+--------+
|nombre|num_dpto| id|nombre_dpto|num_dpto|
+------+--------+---+-----------+--------+
|Sandro|      31| 31|     letras|      31|
| Katia|      33| 33|    derecho|      33|
|  Luis|      33| 33|    derecho|      33|
| Laura|      34| 34| matemática|      34|
|  Raul|      34| 34| matemática|      34|
+------+--------+---+-----------+--------+



In [90]:
df2 = empleados.join(depa, 'num_dpto')

In [91]:
df2.show()

+--------+------+---+-----------+
|num_dpto|nombre| id|nombre_dpto|
+--------+------+---+-----------+
|      31|Sandro| 31|     letras|
|      33| Katia| 33|    derecho|
|      33|  Luis| 33|    derecho|
|      34| Laura| 34| matemática|
|      34|  Raul| 34| matemática|
+--------+------+---+-----------+



In [92]:
df3 = empleados.join(depa, ['num_dpto'])
df3.show()

+--------+------+---+-----------+
|num_dpto|nombre| id|nombre_dpto|
+--------+------+---+-----------+
|      31|Sandro| 31|     letras|
|      33| Katia| 33|    derecho|
|      33|  Luis| 33|    derecho|
|      34| Laura| 34| matemática|
|      34|  Raul| 34| matemática|
+--------+------+---+-----------+



# Shuffle Hash join


In [5]:
empleados.join(broadcast(departamentos), col('num_dpto') == col('id')).show()

+------+--------+---+-----------+
|nombre|num_dpto| id|nombre_dpto|
+------+--------+---+-----------+
|  Luis|      33| 33|    derecho|
| Katia|      33| 33|    derecho|
|  Raul|      34| 34| matemática|
| Laura|      34| 34| matemática|
|Sandro|      31| 31|     letras|
+------+--------+---+-----------+



In [6]:
empleados.join(broadcast(departamentos), col('num_dpto') == col('id')).explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastHashJoin [num_dpto#5L], [id#0L], Inner, BuildRight, false
   :- Filter isnotnull(num_dpto#5L)
   :  +- FileScan parquet [nombre#4,num_dpto#5L] Batched: true, DataFilters: [isnotnull(num_dpto#5L)], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/content/empleados.parquet], PartitionFilters: [], PushedFilters: [IsNotNull(num_dpto)], ReadSchema: struct<nombre:string,num_dpto:bigint>
   +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, bigint, false]),false), [plan_id=107]
      +- Filter isnotnull(id#0L)
         +- FileScan parquet [id#0L,nombre_dpto#1] Batched: true, DataFilters: [isnotnull(id#0L)], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/content/departamentos.parquet], PartitionFilters: [], PushedFilters: [IsNotNull(id)], ReadSchema: struct<id:bigint,nombre_dpto:string>




# EJERCICIO

In [22]:
players = spark.read.option('header','true').option('inferSchema','true').csv('players.csv')
competitions = spark.read.option('header','true').option('inferSchema','true').csv('competitions.csv')
games = spark.read.option('header','true').option('inferSchema','true').csv('games.csv')
clubs = spark.read.option('header','true').option('inferSchema','true').csv('clubs.csv')
appearances = spark.read.option('header','true').option('inferSchema','true').csv('appearances.csv')
leagues = spark.read.option('header','true').option('inferSchema','true').csv('leagues.csv')

In [5]:
players.printSchema()

root
 |-- player_id: integer (nullable = true)
 |-- current_club_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- pretty_name: string (nullable = true)
 |-- country_of_birth: string (nullable = true)
 |-- country_of_citizenship: string (nullable = true)
 |-- date_of_birth: date (nullable = true)
 |-- position: string (nullable = true)
 |-- sub_position: string (nullable = true)
 |-- foot: string (nullable = true)
 |-- height_in_cm: integer (nullable = true)
 |-- market_value_in_gbp: double (nullable = true)
 |-- highest_market_value_in_gbp: double (nullable = true)
 |-- url: string (nullable = true)



In [6]:
players.show()

+---------+---------------+--------------------+--------------------+----------------+----------------------+-------------+----------+------------------+-----+------------+-------------------+---------------------------+--------------------+
|player_id|current_club_id|                name|         pretty_name|country_of_birth|country_of_citizenship|date_of_birth|  position|      sub_position| foot|height_in_cm|market_value_in_gbp|highest_market_value_in_gbp|                 url|
+---------+---------------+--------------------+--------------------+----------------+----------------------+-------------+----------+------------------+-----+------------+-------------------+---------------------------+--------------------+
|    38790|          28095|      dmitri-golubov|      Dmitri Golubov|           UdSSR|                Russia|   1985-06-24|    Attack|    Centre-Forward| Both|         178|               null|                   675000.0|https://www.trans...|
|   106539|          28095|  ale

In [7]:
players.distinct().count()

22428

In [8]:
players.count()

22428

In [16]:
# Determine los tres países con mayor número de jugadores(jugadores nacidos en ese país).
# El resultado debe estar ordenado de forma descendente.

players.groupBy('country_of_birth').agg(count('country_of_birth').alias('players_count')).orderBy(desc('players_count')).show(3)

+----------------+-------------+
|country_of_birth|players_count|
+----------------+-------------+
|          France|         1694|
|           Spain|         1388|
|           Italy|         1312|
+----------------+-------------+
only showing top 3 rows



In [32]:
# Obtenga la lista de jugadores con tarjeta roja. La salida debe contener dos columnas,
# el nombre de pila del jugador y la cantidad de tarjetas rojas que tiene.

players.join(appearances, 'player_id').\
filter(col('red_cards') > 0).\
groupBy(
    'pretty_name').\
    agg(
        count('red_cards').alias('count_red_cards')
        ).\
orderBy(desc('count_red_cards')).\
show()

+--------------------+---------------+
|         pretty_name|count_red_cards|
+--------------------+---------------+
|             Marcelo|              6|
|    Domenico Berardi|              5|
|             Rodrigo|              5|
|        Granit Xhaka|              5|
|          Thomas Lam|              5|
|     Mario Balotelli|              5|
|              Hilton|              5|
|          Joao Pedro|              4|
|          David Luiz|              4|
|      Rafik Halliche|              4|
|   Kalidou Koulibaly|              4|
|          Ante Rebic|              4|
|     Yannick Cahuzac|              4|
|     Damien Da Silva|              4|
|       Ramon Leeuwin|              4|
|Timothee Kolodzie...|              4|
|     Roberto Soriano|              4|
|     Rodrigo De Paul|              4|
|         Ivan Ordets|              4|
|     Stefan Mitrovic|              4|
+--------------------+---------------+
only showing top 20 rows



In [106]:
# ¿Cuántos juegos se jugaron en la Premier League? La salida debe contener dos columnas,
# el nombre de la liga y la cantidad de juegos que se jugaron en ella.

gam_comp = games.join(competitions, col('competition_id') == col('competition_code'))

In [112]:
gam_comp.filter(col('name') == 'premier-league').groupBy('name').agg(count('game_id').alias('juegos_jugados')).show()

+--------------+--------------+
|          name|juegos_jugados|
+--------------+--------------+
|premier-league|          2809|
+--------------+--------------+



In [115]:
# Obtenga las tres ligas con mayor número de asistencia de público teniendo en cuenta todos los juegos que se jugaron en ellas.
# El resultado debe estar ordenado de forma descendente y tener dos columnas, el nombre de la liga y la asistencia total.

gam_comp.groupBy('name').agg(sum('attendance').alias('audiencia')).orderBy(desc('audiencia')).limit(3).show()

+--------------+---------+
|          name|audiencia|
+--------------+---------+
|premier-league| 86964852|
|    bundesliga| 78102473|
|        laliga| 62943533|
+--------------+---------+



In [116]:
competitions.show()

+--------------+--------------------+------------------+----------+------------+--------------------+-------------+--------------------+
|competition_id|                name|              type|country_id|country_name|domestic_league_code|confederation|                 url|
+--------------+--------------------+------------------+----------+------------+--------------------+-------------+--------------------+
|            L1|          bundesliga|        first_tier|        40|     Germany|                  L1|       europa|https://www.trans...|
|           DFB|           dfb-pokal|      domestic_cup|        40|     Germany|                  L1|       europa|https://www.trans...|
|           DFL|        dfl-supercup|domestic_super_cup|        40|     Germany|                  L1|       europa|https://www.trans...|
|           NL1|          eredivisie|        first_tier|       122| Netherlands|                 NL1|       europa|https://www.trans...|
|           NLP|     toto-knvb-beker|    